In [1]:
#Data Manipulation
import numpy as np
import pandas as pd

#Plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns #Some shiny visualizations

#Utilities
import json
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, f1_score, recall_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


#Artificial Neural Networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Activation, Dropout, BatchNormalization

Using TensorFlow backend.


In [2]:
#--------- Project Administration ------------
DatasetFilepath = './SLE_RA_Control_blood_panels.csv'
#--------------------------------------------

In [4]:
data = pd.read_csv(DatasetFilepath)
data = data.drop(data.columns[0], axis=1)

In [5]:
# Create correlation matrix
corr_matrix = data.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
data.drop(to_drop, axis=1, inplace=True)

In [6]:
data.to_csv('CORE_SLE_RA_Control_blood_panels.csv')

In [177]:
label = LabelEncoder()
int_data = label.fit_transform(data['Condition'])
int_data = int_data.reshape(len(int_data), 1)

onehot_data = OneHotEncoder(sparse=False)
onehot_data = onehot_data.fit_transform(int_data)

y = onehot_data

In [178]:
x = data.drop(['Name', 'Condition'], axis=1)

In [179]:
# Prepare and scale the data. 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=25)

x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

In [180]:
#--------------------------------------------------------------------------
def make_that_model(layers, loss_function, optimizer, learning_rate, exit_activation=None):
    model = keras.Sequential()
    for layer in layers:
        model.add(getattr(keras.layers, layer["name"])(**layer["options"]))

    if exit_activation!=None:
        model.add(Activation(exit_activation))

    op = getattr(keras.optimizers, optimizer)(learning_rate=(learning_rate))
    model.compile(loss=loss_function, optimizer=op, metrics=['accuracy'])

    return model
#--------------------------------------------------------------------------
def model_create(parameterization, load_file=None):

  model = make_that_model(
      parameterization.get('layers'),
      parameterization.get('loss_function'),
      parameterization.get('optimizer'),
      parameterization.get('learning_rate'),
      )
  
  if load_file != None:
    model.load_weights(load_file)
  
  return model
#--------------------------------------------------------------------------

In [267]:
config = {
    "layers": [
        {
            "name": "Dense",
            "options": {
                "input_shape": (x.shape[1],),
                "units": 173,
                "activation":"relu", 
                "use_bias": True,
            },
        },
        {
            "name": "Dropout",
            "options": {
                "rate": 0.2,
            },
        },
        {
            "name": "Dropout",
            "options": {
                "rate": 0.2,
            },
        },
        {
            "name": "Dense",
            "options": {
                "units": 4,
                "use_bias": True,
            },
        },
    ],
    "loss_function": "categorical_crossentropy",
    "optimizer": "Adam",
    "batch_size": 512,
    "learning_rate": 0.000075,
}

In [268]:
model = model_create(config)
model.summary()

TypeError: ('Keyword argument not understood:', 'units')

In [269]:
hist = model.fit(
        x=x_train, y=y_train, 
        validation_data=(x_test,y_test),
        validation_freq=5,
        workers=8,
        use_multiprocessing=True,
        epochs=100, 
        verbose=1,
        )

Epoch 1/100
2/2 [==============================] - 0s 3ms/step - loss: 4.9206 - accuracy: 0.4583
Epoch 2/100
2/2 [==============================] - 0s 3ms/step - loss: 4.5437 - accuracy: 0.4375
Epoch 3/100
2/2 [==============================] - 0s 4ms/step - loss: 4.5818 - accuracy: 0.4375
Epoch 4/100
2/2 [==============================] - 0s 3ms/step - loss: 4.8835 - accuracy: 0.4375
Epoch 5/100
2/2 [==============================] - 0s 71ms/step - loss: 4.2211 - accuracy: 0.4167 - val_loss: 10.8298 - val_accuracy: 0.6667
Epoch 6/100
2/2 [==============================] - 0s 5ms/step - loss: 4.9815 - accuracy: 0.4167
Epoch 7/100
2/2 [==============================] - 0s 5ms/step - loss: 4.8354 - accuracy: 0.4583
Epoch 8/100
2/2 [==============================] - 0s 6ms/step - loss: 4.1453 - accuracy: 0.4375
Epoch 9/100
2/2 [==============================] - 0s 5ms/step - loss: 4.1722 - accuracy: 0.4375
Epoch 10/100
2/2 [==============================] - 0s 40ms/step - loss: 4.1551 - a

2/2 [==============================] - 0s 4ms/step - loss: 5.7206 - accuracy: 0.5000
Epoch 79/100
2/2 [==============================] - 0s 3ms/step - loss: 5.3918 - accuracy: 0.5208
Epoch 80/100
2/2 [==============================] - 0s 31ms/step - loss: 5.0757 - accuracy: 0.5833 - val_loss: 8.3563 - val_accuracy: 0.7500
Epoch 81/100
2/2 [==============================] - 0s 3ms/step - loss: 7.4125 - accuracy: 0.4167
Epoch 82/100
2/2 [==============================] - 0s 3ms/step - loss: 5.0443 - accuracy: 0.5208
Epoch 83/100
2/2 [==============================] - 0s 6ms/step - loss: 6.7195 - accuracy: 0.5208
Epoch 84/100
2/2 [==============================] - 0s 6ms/step - loss: 6.3801 - accuracy: 0.6042
Epoch 85/100
2/2 [==============================] - 0s 47ms/step - loss: 5.7182 - accuracy: 0.5417 - val_loss: 8.3279 - val_accuracy: 0.7500
Epoch 86/100
2/2 [==============================] - 0s 5ms/step - loss: 8.0729 - accuracy: 0.5417
Epoch 87/100
2/2 [===========================

In [208]:
pred = model.predict(x_test)

In [242]:
batch_size=30


In [249]:
stat = classification_report(np.argmax(pred,-1), np.argmax(y_test, -1), output_dict=True)
metrics = dict(
        zip(model.metrics_names,
            model.evaluate(x=x_test, y=y_test, batch_size=batch_size)))

1/1 [==============================] - 0s 16ms/step - loss: 2.6863 - accuracy: 0.5833


In [251]:
metrics

{'loss': 2.6863491535186768, 'accuracy': 0.5833333134651184}

In [250]:
stat

{'0': {'precision': 1.0,
  'recall': 0.5714285714285714,
  'f1-score': 0.7272727272727273,
  'support': 7},
 '1': {'precision': 0.6,
  'recall': 1.0,
  'f1-score': 0.7499999999999999,
  'support': 3},
 '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2},
 'accuracy': 0.5833333333333334,
 'macro avg': {'precision': 0.5333333333333333,
  'recall': 0.5238095238095238,
  'f1-score': 0.4924242424242424,
  'support': 12},
 'weighted avg': {'precision': 0.7333333333333334,
  'recall': 0.5833333333333334,
  'f1-score': 0.6117424242424242,
  'support': 12}}

In [253]:
label.inverse_transform(np.argmax(y_test, axis=-1))
label.inverse_transform(np.argmax(pred, axis=-1))

array(['disease: SLE (Systemic LUPUS Erythomatosus)',
       'disease: Rheumatoid Arthiritis (DMARD-IR)',
       'disease: Rheumatoid Arthiritis (DMARD-IR)', 'disease: Control',
       'disease: Rheumatoid Arthiritis (DMARD-IR)', 'disease: Control',
       'disease: Control', 'disease: Control', 'disease: Control',
       'disease: Control', 'disease: Control',
       'disease: SLE (Systemic LUPUS Erythomatosus)'], dtype=object)